In [1]:
# pip install boto3
# pip install python-dotenv
# pip install trino
# pip install pandas
# pip install pyarrow
# pip install --upgrade sqlalchemy==1.3 sqlalchemy-trino

In [1]:
import boto3
import pandas as pd
import io

Load Environment Variables

In [2]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Create an S3 resource through AWS client access, retrieve the relevant bucket and list objects

In [3]:
s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_ENDPOINT'],
    aws_access_key_id=os.environ['S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_SECRET_KEY'],
)

In [4]:
bucket = s3_resource.Bucket(os.environ['S3_BUCKET'])

# list all objects
#for obj in bucket.objects.all():
#    print(obj.key, obj.last_modified)
    
files = []
for file in bucket.objects.filter(Prefix='PUDL/pudl-0.4.0/pudl_data/parquet/'):
    if file.key.endswith('parquet'):
        files.append(file.key)
files[0:10]

['PUDL/pudl-0.4.0/pudl_data/parquet/epacems/year=1995/state=AL/0bd3b8d4be104c6c9b4531fff26f0671.parquet',
 'PUDL/pudl-0.4.0/pudl_data/parquet/epacems/year=1995/state=FL/a3c8fed0646345c5a959862f6307112e.parquet',
 'PUDL/pudl-0.4.0/pudl_data/parquet/epacems/year=1995/state=GA/7f6fa554bac640bc8ac6b4f7698f1e22.parquet',
 'PUDL/pudl-0.4.0/pudl_data/parquet/epacems/year=1995/state=IA/8cae277530824844ad4f54b7c88e8813.parquet',
 'PUDL/pudl-0.4.0/pudl_data/parquet/epacems/year=1995/state=IL/4854d33914df43cf9b45527d1c56879c.parquet',
 'PUDL/pudl-0.4.0/pudl_data/parquet/epacems/year=1995/state=IN/2f40504880c74aea9884d2e7576ef89e.parquet',
 'PUDL/pudl-0.4.0/pudl_data/parquet/epacems/year=1995/state=KS/9aed897519164b20b8346c15353d892d.parquet',
 'PUDL/pudl-0.4.0/pudl_data/parquet/epacems/year=1995/state=KY/793ec765eb474844a92bf2821b4f9d78.parquet',
 'PUDL/pudl-0.4.0/pudl_data/parquet/epacems/year=1995/state=MA/e8044aec4c784e3e84031b233a32e037.parquet',
 'PUDL/pudl-0.4.0/pudl_data/parquet/epacems/ye

Read the first parquet file into a Pandas data frame and view the results

In [5]:
buffer = io.BytesIO()
parquet_file = s3_resource.Object(os.environ['S3_BUCKET'],files[0])
parquet_file.download_fileobj(buffer)
df = pd.read_parquet(buffer)
df

,plant_id_eia,unitid,operating_datetime_utc,operating_time_hours,gross_load_mw,steam_load_1000_lbs,so2_mass_lbs,so2_mass_measurement_code,nox_rate_lbs_mmbtu,nox_rate_measurement_code,nox_mass_lbs,nox_mass_measurement_code,co2_mass_tons,co2_mass_measurement_code,heat_content_mmbtu,facility_id,unit_id_epa
0,7,1,1995-01-01 06:00:00+00:00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,<NA>,<NA>
1,7,1,1995-01-01 07:00:00+00:00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,<NA>,<NA>
2,7,1,1995-01-01 08:00:00+00:00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,<NA>,<NA>
3,7,1,1995-01-01 09:00:00+00:00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,<NA>,<NA>
4,7,1,1995-01-01 10:00:00+00:00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122635,56,3,1996-01-01 01:00:00+00:00,1.0,198.0,NaN,2030.500000,Measured,0.514,Measured,985.749023,Calculated,196.800003,Measured,1917.800049,11,55
122636,56,3,1996-01-01 02:00:00+00:00,1.0,176.0,NaN,1629.599976,Measured,0.534,Measured,906.731995,Calculated,174.199997,Measured,1698.000000,11,55
122637,56,3,1996-01-01 03:00:00+00:00,1.0,167.0,NaN,1495.400024,Measured,0.539,Measured,869.083984,Calculated,165.399994,Measured,1612.400024,11,55
122638,56,3,1996-01-01 04:00:00+00:00,1.0,174.0,NaN,1630.800049,Measured,0.536,Measured,892.921997,Calculated,170.899994,Measured,1665.900024,11,55


In [6]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122640 entries, 0 to 122639
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype              
---  ------                     --------------   -----              
 0   plant_id_eia               122640 non-null  int32              
 1   unitid                     122640 non-null  string             
 2   operating_datetime_utc     122640 non-null  datetime64[ns, UTC]
 3   operating_time_hours       122639 non-null  float32            
 4   gross_load_mw              122640 non-null  float32            
 5   steam_load_1000_lbs        0 non-null       float32            
 6   so2_mass_lbs               101291 non-null  float32            
 7   so2_mass_measurement_code  101292 non-null  category           
 8   nox_rate_lbs_mmbtu         101291 non-null  float32            
 9   nox_rate_measurement_code  101292 non-null  category           
 10  nox_mass_lbs               100628 non-null  float32     

In [13]:
import trino
JWT_TOKEN = os.environ['TRINO_PASSWD']
conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(JWT_TOKEN),
    verify=True,
)
cur = conn.cursor()

In [15]:
_p2smap = {
    'string': 'varchar',
    'float32': 'double',
    'int32': 'bigint',
    'Int32': 'bigint',
    'category': 'varchar',
    'datetime64[ns, UTC]': 'timestamp'
}

def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))

# add ability to specify optional dict for specific fields?
# if column name is present, use specified value?
def generate_table_schema_pairs(df):
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0],t=e[1]) for e in pz])

In [16]:
tablename = 'Y95_AL'
df.to_parquet('/tmp/{tname}.parquet'.format(tname=tablename), index=False)

In [10]:
# Create an S3 client
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_ENDPOINT'],
    aws_access_key_id=os.environ['S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_SECRET_KEY'],
)

In [23]:
s3.upload_file(
    Bucket=os.environ['S3_BUCKET'],
    Key='trino/pudl/epacems/{tname}/{tname}.parquet'.format(tname=tablename),
    Filename='/tmp/{tname}.parquet'.format(tname=tablename)
)

In [25]:
schema = generate_table_schema_pairs(df)

tabledef = """create table if not exists osc_physical_landing.epacems.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/pudl/epacems/{tname}/'
)""".format(schema=schema,bucket=os.environ['S3_BUCKET'],tname=tablename)
print(tabledef)

# tables created externally may not show up immediately in cloud-beaver
cur.execute(tabledef)
cur.fetchall()

create table if not exists osc_physical_landing.epacems.Y95_AL(
    plant_id_eia bigint,
    unitid varchar,
    operating_datetime_utc timestamp,
    operating_time_hours double,
    gross_load_mw double,
    steam_load_1000_lbs double,
    so2_mass_lbs double,
    so2_mass_measurement_code varchar,
    nox_rate_lbs_mmbtu double,
    nox_rate_measurement_code varchar,
    nox_mass_lbs double,
    nox_mass_measurement_code varchar,
    co2_mass_tons double,
    co2_mass_measurement_code varchar,
    heat_content_mmbtu double,
    facility_id bigint,
    unit_id_epa bigint
) with (
    format = 'parquet',
    external_location = 's3a://redhat-osc-physical-landing-647521352890/trino/pudl/epacems/Y95_AL/'
)


TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot create table osc_physical_landing.epacems.y95_al", query_id=20210928_211107_00002_gbb2e)